# 推定

母集団のパラメタを推定する<br>
単純な点推定を説明した後、推定誤差を加味した区間推定の方法を説明

In [28]:
# 数値計算に使うライブラリ
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats



# グラフを描画するライブラリ
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

# 表示桁数の指定
%precision 3
# グラフをjupyter Notebook内に表示させるための指定
%matplotlib inline

The directory '/home/yu/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/yu/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
  Could not find a version that satisfies the requirement install (from versions: )
No matching distribution found for install
You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


ModuleNotFoundError: No module named 'cv2'

In [2]:
# データの読み込み
fish = pd.read_csv("sample/3-7-1-fish_length.csv")["length"]
fish

0    4.352982
1    3.735304
2    5.944617
3    3.798326
4    4.087688
5    5.265985
6    3.272614
7    3.526691
8    4.150083
9    3.736104
Name: length, dtype: float64

## 点推定

点推定は母数をある１つの値として指定する推定方法<br>
母平均を推定する場合は、標本平均を推定量として使う<br>
<br>
早い話が標本から平均値を求めていく<br>
標本平均は普偏性も一致性も満たす統計量であるため、母平均の推定値として望ましい

In [3]:
mu = sp.mean(fish)
mu

4.187

標本平均が4.187なので母平均も4.187だろうと推定する<br>
これが点推定

母分散の推定値としては不偏分散を用いる

In [4]:
sigma2 = sp.var(fish,ddof=1)
sigma2

0.680

## 区間推定

推定値に幅をもたせた推定方法<br>
幅をもたせることで推定誤差を加味することができる<br>
誤差が小さければ区間推定の幅は狭く、サンプルサイズが大きくても動揺に区間推定の幅は狭い

## 信頼係数・区間

信頼係数　：　区間推定の幅における信頼度合い<br>
信頼区間　：　ある信頼係数を満たす区間<br>
<br>
同じデータを対象とした場合は、信頼係数が大きいほど信頼区間の幅は広くなる<br>
信頼の度合いをあげようとすると、安全第一で幅を広く取らざるを得ない

## 信頼限界

信頼区間の上限および下限値<br>

## 区間推定と信頼区間(stats.interval関数)

区間推定の際にはt分布のパーセント点を用いる<br>
信頼係数を95%とするなら、t分布における2.5%点と97.5%点を計算する<br>
t分布に従う変数がこの区間に入る確率は97.5-2.5=95%ということになる
<br><br>
区間推定に必要な情報は、自由度（サンプルサイズ-1）,標本平均,標準誤差<br>

In [5]:
#自由度
df = len(fish)-1
df

9

In [7]:
#標準誤差
sigma = sp.std(fish,ddof = 1)
se = sigma / sp.sqrt(len(fish))
se

0.2608259396768776

信頼区間は、信頼係数alpha,自由度df,標本平均loc,標準誤差scaleを指定する<br>
出力の１つ目が下側信頼限界<br>
2つ目が上側信頼限界

In [8]:
interval = stats.t.interval(
    alpha = 0.95,df = df,loc=mu,scale=se
)
interval

(3.597010056835825, 4.777068592173221)

95%信頼区間は3.597から4.777となる

## 信頼区間（詳細）

1. ある自由度を持つt分布における97.5%点を計算する<br>
 1.1 t分布における97.5%点を$t_{0.975}$と表記する<br>
 1.2 t分布は平均にたいして左右対称であるため、2.5%点は$-t_{0.975}$として計算できる<br>
 1.3 t分布に従う変数が$-t_{0.975}$以上、$t_{0.975}$以下になる確率は95%である<br>
  1.3.1 このときの95%が信頼区間となる<br>
2. 標本平均$-t_{0.975}$ * 標準誤差　が下側信頼限界
3. 標本平均$+t_{0.975}$ * 標準誤差　が上側信頼限界
<br><br><br>
<u>なぜ97.5%なのかというと、t分布は標準正規分布とほぼ同じなので左右対称の形をしている<br>
そのなかで95%存在する確率は、両端から合計5%の区間を除いたすべてと考えるので<br>
5/2=2.5%よって2.5%点と100-2.5=97.5%のパーセント点を求めている</u>


## 信頼区間の幅を決める要素

標本における分散が大きければ、<br>
「データが平均値から離れている→平均値をあまり信用できない」<br>
ことになるので信頼区間の幅が広くなる<br>
<br>

In [9]:
# 標本標準偏差が大きいと、信頼区間は広くなる
se2 = (sigma*10) / sp.sqrt(len(fish))
stats.t.interval(
    alpha = 0.95, df = df, loc = mu, scale = se2)

(-1.7132533521824618, 10.087332001191509)

標本標準偏差を10倍に増やして動揺に95%点を計算すると、<br>
信頼区間の幅が広がっていることがわかる<br>
「真の母平均がどこに位置しているのかがよくわからない」ということ<br><br>
逆にサンプルサイズが大きくなれば、標本平均を信頼できるようになるため、信頼区間は狭くなる<br>


In [10]:
# サンプルサイズが大きいと、信頼区間は狭くなる
df2 = (len(fish)*10) - 1
se3 = sigma / sp.sqrt(len(fish)*10)
stats.t.interval(
    alpha = 0.95, df = df2, loc = mu, scale = se3)

(4.0233803082774395, 4.350698340731607)

まったく同一のデータであった場合、信頼係数が大きいほど、安全を見込んで信頼区間の幅は広く取られる<br>
一例として、99%信頼区間の幅は95%よりも広く取られる

In [11]:
stats.t.interval(
    alpha = 0.99, df = df, loc = mu, scale = se)

(3.3393979149413977, 5.034680734067649)

## 区間推定の結果の解釈

信頼係数95%の95%とは以下のようにして得られる<br>
1. 真の母集団分布から標本を抽出する
2. 今回と同じやり方で95%信頼区間を計算する
3. この試行をたくさん繰り返す
4. すべての試行のうち、「真の母数」が信頼区間に含まれている割合が95%



![alt text](7th.jpg "Title")

In [29]:
# 信頼区間が母平均(4)を含んでいればTrue
be_included_array = np.zeros(20000, dtype = "bool")
be_included_array

array([False, False, False, ..., False, False, False])

In [30]:
# 「データを10個選んで95%信頼区間を求める」試行を20000回繰り返す
# 信頼区間が母平均(4)を含んでいればTrue
np.random.seed(1)
norm_dist = stats.norm(loc = 4, scale = 0.8)
for i in range(0, 20000):
    sample = norm_dist.rvs(size = 10)
    df = len(sample) - 1
    mu = sp.mean(sample)
    std = sp.std(sample, ddof = 1)
    se = std / sp.sqrt(len(sample))
    interval = stats.t.interval(0.95, df, mu, se)
    if(interval[0] <= 4 and interval[1] >= 4):
        be_included_array[i] = True

信頼区間が母平均を含んでいた割合はおおよそ0.95となる

In [31]:
sum(be_included_array) / len(be_included_array)

0.948